In [1]:
pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
#################################################
# correct drifts in the "dynamics movie"
import canal.image.io

In [3]:
# create loader
filename = '/media/ytyoonubt/Analysis161022/161022/s00/161022s00t00_00.nii'
movieloader = canal.image.io.Nifti.loader(filename)
movieloader.shape

(32736, 1024, 2048)

In [4]:
# load movie and reshape into (t, z, y, x)
n_zplanes = 32
movie = movieloader.load().reshape((-1, n_zplanes) + movieloader.shape[-2:])
movie.shape

(1023, 32, 1024, 2048)

In [5]:
# max projection along the z-axis
proj = np.empty((len(movie),) + movie.shape[-2:], movie.dtype)
movie.max(axis=1, out=proj)

array([[[157, 190, 173, ..., 198, 152, 160],
        [118, 119, 111, ..., 109, 112, 116],
        [114, 105, 106, ..., 115, 107, 107],
        ..., 
        [109, 109, 114, ..., 120, 117, 106],
        [113, 166, 114, ..., 113, 123, 116],
        [148, 172, 177, ..., 130, 192, 183]],

       [[192, 185, 176, ..., 143, 145, 157],
        [113, 116, 110, ..., 108, 109, 117],
        [110, 107, 108, ..., 113, 108, 106],
        ..., 
        [108, 113, 114, ..., 116, 110, 106],
        [112, 114, 113, ..., 113, 113, 115],
        [153, 149, 154, ..., 161, 177, 131]],

       [[144, 165, 159, ..., 169, 152, 165],
        [115, 107, 116, ..., 121, 112, 110],
        [106, 110, 109, ..., 115, 108, 107],
        ..., 
        [108, 109, 110, ..., 113, 108, 106],
        [124, 111, 109, ..., 109, 120, 113],
        [142, 129, 164, ..., 137, 152, 140]],

       ..., 
       [[114, 113, 110, ..., 107, 109, 106],
        [113, 106, 106, ..., 108, 115, 110],
        [106, 107, 109, ..., 117, 111, 

In [6]:
import canal.image.registration

In [7]:
# create feature detector
feature_shape = (256, 256)
feature_strides = (32, 32)
detector = canal.image.registration.FeatureDetector(feature_shape, feature_strides,
                                                    n_keypoints=256)

In [8]:
# detect features
template = proj[500]
keypoints, features = detector.detect(template, verbose=True)

  1%|          | 7/833 [00:00<00:11, 69.99feature/s]

Inspecting features


100%|██████████| 833/833 [00:27<00:00, 30.62feature/s]


In [9]:
# create feature tracker
tracker = canal.image.registration.FeatureTracker(keypoints, features)

In [10]:
# track
valid_from = 50 # wait until photo-bleach stabilized
trajectory = tracker.track(proj[valid_from:], tolerance=32, verbose=True)

  3%|▎         | 7/256 [00:00<00:03, 64.13feature/s]

Initial search


  0%|          | 0/973 [00:00<?, ?frame/s]

Tracking


100%|██████████| 973/973 [41:08<00:00,  2.54s/frame]


In [11]:
# create mapper
offsets = canal.image.registration.create_transform(trajectory, keypoints)
mapper = canal.image.registration.MovieMapper(movie[valid_from:], offsets)

In [12]:
print('{} -> {}'.format(movie[valid_from:].shape, mapper.shape))
# the drift correction ended
#########################

(973, 32, 1024, 2048) -> (973, 32, 1003, 2012)


In [13]:
##################################################################################
# 1. detect cells in the "reference movie"
# 2. perform a registration between the "dynamics movie" and the "intermediate movie"
# 3. extract signals
import canal.cell

In [14]:
# load the movie including the "reference movie" and the "intermediate movie"
filename = '/media/ytyoonubt/Analysis161022/161022/s00/161022s00ref.nii'
imageloader = canal.image.io.Nifti.loader(filename)
imageloader.shape

(34000, 1024, 2048)

In [15]:
# reshape the movie into (t, c, z, y, x)
n_channels = 2
n_dense_zplanes = 1000
image = imageloader.load().reshape((-1, n_channels, n_dense_zplanes) + 
                                   imageloader.shape[-2:])
image.shape

(17, 2, 1000, 1024, 2048)

In [16]:
# copy image
reference = np.empty(image.shape[-3:], image.dtype)
reference[:] = image[8, 1]
intermediate = np.empty_like(reference)
intermediate[:] = image[8, 0]

In [20]:
# trim image
# the images from okazaki have artifacts in column number 3 :(
trim_from = 4

In [24]:
# find cells
nuclei_rad = 10 # radius of the nuclei in pixels
cpu_partition = (2, 4, 4)
cells = canal.cell.find_cells(reference[..., trim_from:], nuclei_rad, cpu_partition)

In [27]:
# get a map between the "dynamics movie" and the "intermediate movie"
embedder = canal.image.registration.Embedder(intermediate[..., trim_from:])
z_ratio = 17
transform = embedder.embed(mapper[-1], z_ratio, verbose=True)

  0%|          | 9/2914 [00:00<00:35, 82.97feature/s]

Inspecting features


 10%|█         | 294/2914 [00:00<00:00, 2937.08point/s]

Checking distances between the key points


  3%|▎         | 8/256 [00:00<00:03, 69.74feature/s]

...

  3%|▎         | 29/1032 [00:00<00:03, 281.62frame/s]

XY shift detected: [-10.  -4.]
Creating movie


  0%|          | 25/32000 [00:00<02:08, 249.57pair/s]

Measuring distances


100%|██████████| 32000/32000 [01:52<00:00, 284.79pair/s]


In [29]:
# extract signals from the movie
valid_cells, signals = canal.cell.extract(mapper, cells, nuclei_rad, transform, verbose=True)
# 1, 2, 3 ended
################

  7%|▋         | 937/14033 [00:00<00:03, 4225.67cell/s]

Creating masks


  0%|          | 1/973 [00:00<02:51,  5.66frame/s]

Extracting signals: 5318 cells


100%|██████████| 973/973 [01:52<00:00,  8.65frame/s]


In [34]:
# normalize the signals
import canal.signal
time_from = 100 # steep photobleaching
act_time_scale = 7 # time scale of neural activity
normalized = canal.signal.normalize(signals, act_time_scale)

In [37]:
imshow(normalized, aspect='auto')
# normalization ended
#####################